## Import Libraries

In [ ]:
import cv2
import os
import numpy as np
from skimage.feature import greycomatrix, greycoprops
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

## Import Env File

In [ ]:
from dotenv import load_dotenv  

load_dotenv()
datasets_path = os.getenv('DATASET_PATH_CONDITIONS_2')
models_path = os.getenv('MODELS_PATH')

## Parameters

In [ ]:
classes = ['class_1', 'class_2', 'class_3', 'class_4', 'class_5']
image_size = (224, 224)  # Resize images to a fixed size

## GLCM Function

In [ ]:
def extract_glcm_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    glcm = greycomatrix(gray, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = greycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = greycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = greycoprops(glcm, 'homogeneity')[0, 0]
    energy = greycoprops(glcm, 'energy')[0, 0]
    correlation = greycoprops(glcm, 'correlation')[0, 0]
    return [contrast, dissimilarity, homogeneity, energy, correlation]

## Load The Dataset and Extract The Features

In [ ]:
features = []
labels = []
for class_idx, class_name in enumerate(classes):
    folder = f'path_to_train_data/{class_name}'
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        image = cv2.imread(img_path)
        image = cv2.resize(image, image_size)
        glcm_features = extract_glcm_features(image)
        features.append(glcm_features)
        labels.append(class_idx)
        
features = np.array(features)
labels = np.array(labels)

## Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

## Train SVM

In [ ]:
clf = SVC(kernel='linear', probability=True)
clf.fit(X_train, y_train)

## Evaluates

In [ ]:
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

## Save The Model

In [ ]:
from joblib import dump

# Simpan model ke file
dump(clf, 'svm_glcm_model.joblib')
print("Model berhasil disimpan ke svm_glcm_model.joblib")